In [20]:
import pandas as pd
import numpy as np

- **Problem Statement**: How does women's representation in the government impact various economic and social factors (like education, GDP, healthcare, population growth, employment) of a country/continent? 
- **Data**: Use the required datasets (for education, GDP, healthcare, population growth, employment etc)) from the http://data.un.org/

- **Implementation Approach**:
    1. Use python code to clean, merge and prepare the dataset.
    2. Import the data set to GCP's BigQuery
    3. Identify & define new meaningful variables (ratios etc) that will be useful for the defined problem statement
    4. Create dashboards in Looker Studio to show how the various factors influence the presence of women in govt.

- Overall - world - all/ year
- Region          - all/ year
- Country         - all/ year
- Top 5 & Bottom 5

## Read the data

In [21]:
CSVs = [ "Seats held by women in Parliament.csv", 
        "Population, Surface Area and Density.csv" , 
        "GDP and GDP Per Capita.csv",
        "Education.csv",
        "Labour Force and Unemployment.csv"]
COLS = ["Region/Country/Area", "Year", "Series", "Value"]
COLS_INDEX = ['Region/Country/Area', 'Year']
DFs = {}

df_regions = pd.read_csv("data/Regions.csv")
display(df_regions[:5])

for csv in CSVs:
    DFs[csv] = pd.read_csv(f"data/{csv}")[COLS]
    print(DFs[csv]['Value'].dtype)
    if  DFs[csv]['Value'].dtype == 'object':
        DFs[csv]['Value'] = DFs[csv]['Value'].str.replace(',','')
    DFs[csv]['Value'] = DFs[csv]['Value'].astype(float)
    display(DFs[csv][:5])

,Region,Type
0,Africa,Region
1,Americas,Region
2,Asia,Region
3,Australia and New Zealand,Region
4,Caribbean,Region


float64


,Region/Country/Area,Year,Series,Value
0,"Total, all countries or areas",2000,"Seats held by women in national parliament, as...",13.3
1,"Total, all countries or areas",2005,"Seats held by women in national parliament, as...",15.9
2,"Total, all countries or areas",2010,"Seats held by women in national parliament, as...",19.0
3,"Total, all countries or areas",2015,"Seats held by women in national parliament, as...",22.3
4,"Total, all countries or areas",2018,"Seats held by women in national parliament, as...",23.4


object


,Region/Country/Area,Year,Series,Value
0,"Total, all countries or areas",2010,Population mid-year estimates (millions),6985.60
1,"Total, all countries or areas",2010,Population mid-year estimates for males (milli...,3514.41
2,"Total, all countries or areas",2010,Population mid-year estimates for females (mil...,3471.20
3,"Total, all countries or areas",2010,Sex ratio (males per 100 females),101.20
4,"Total, all countries or areas",2010,Population aged 0 to 14 years old (percentage),27.10


...

,Region/Country/Area,Year,Series,Value
0,"Total, all countries or areas",1995,GDP in current prices (millions of US dollars),31247262.0
1,"Total, all countries or areas",2005,GDP in current prices (millions of US dollars),47730924.0
2,"Total, all countries or areas",2010,GDP in current prices (millions of US dollars),66461443.0
3,"Total, all countries or areas",2015,GDP in current prices (millions of US dollars),75133208.0
4,"Total, all countries or areas",2018,GDP in current prices (millions of US dollars),86357998.0


object


,Region/Country/Area,Year,Series,Value
0,"Total, all countries or areas",2005,Students enrolled in primary education (thousa...,679014.0
1,"Total, all countries or areas",2005,Gross enrollment ratio - Primary (male),104.5
2,"Total, all countries or areas",2005,Gross enrollment ratio - Primary (female),99.7
3,"Total, all countries or areas",2005,Students enrolled in secondary education (thou...,509277.0
4,"Total, all countries or areas",2005,Gross enrollment ratio - Secondary (male),65.8


float64


,Region/Country/Area,Year,Series,Value
0,"Total, all countries or areas",2005,Labour force participation - Total,63.7
1,"Total, all countries or areas",2005,Unemployment rate - Total,5.9
2,"Total, all countries or areas",2005,Labour force participation - Male,77.0
3,"Total, all countries or areas",2005,Unemployment rate - Male,5.7
4,"Total, all countries or areas",2005,Labour force participation - Female,50.5


## Merge the data to create summary

In [22]:
df_all = pd.concat(list(DFs.values())).sort_values(by=COLS_INDEX)
df_all = df_all.sort_values(by=COLS_INDEX)
[print(csv, df.shape[0]) for csv, df in DFs.items()]
df_lengths = [df.shape[0] for df in list(DFs.values())]

import functools
print(functools.reduce(lambda l1, l2: l1 + l2, df_lengths))

print(df_all.shape[0])
df_all.to_csv("out/all.csv", index=False)
df_all[:5]

Seats held by women in Parliament.csv 1953
Population, Surface Area and Density.csv 7873
GDP and GDP Per Capita.csv 6786
Education.csv 7282
Labour Force and Unemployment.csv 5475
29369
29369


,Region/Country/Area,Year,Series,Value
840,Afghanistan,1995,GDP in current prices (millions of US dollars),2757.0
847,Afghanistan,1995,GDP per capita (US dollars),152.0
854,Afghanistan,1995,GDP in constant 2010 prices (millions of US do...,6335.0
861,Afghanistan,1995,GDP real rates of growth (percent),30.5
841,Afghanistan,2005,GDP in current prices (millions of US dollars),6221.0


In [23]:
df_summary = pd.pivot_table(data=df_all, index=COLS_INDEX, columns='Series', values='Value', aggfunc=np.average)
df_summary = df_summary.reset_index()
df_summary = pd.merge(df_regions, df_summary, how='inner',left_on='Region', right_on='Region/Country/Area')
df_summary = df_summary.drop(columns='Region/Country/Area')
df_summary.to_csv("out/summary.csv", index=False)
df_summary.shape, df_summary.columns, display(df_summary)

,Region,Type,Year,GDP in constant 2010 prices (millions of US dollars),GDP in current prices (millions of US dollars),GDP per capita (US dollars),GDP real rates of growth (percent),Gross enrollment ratio - Primary (female),Gross enrollment ratio - Primary (male),Gross enrollment ratio - Secondary (female),...,Population mid-year estimates for males (millions),"Seats held by women in national parliament, as of February (%)",Sex ratio (males per 100 females),Students enrolled in primary education (thousands),Students enrolled in secondary education (thousands),Students enrolled in upper secondary education (thousands),Surface area (thousand km2),Unemployment rate - Female,Unemployment rate - Male,Unemployment rate - Total
0,Africa,Region,1995,963827.0,588279.0,821.0,2.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa,Region,2005,1502529.0,1133617.0,1239.0,6.2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.6,6.7,7.1
2,Africa,Region,2010,1953445.0,1979101.0,1907.0,5.9,NaN,NaN,NaN,...,525.87,NaN,99.3,NaN,NaN,NaN,NaN,7.4,5.9,6.6
3,Africa,Region,2015,2358404.0,2358404.0,1997.0,3.4,NaN,NaN,NaN,...,599.30,NaN,99.6,NaN,NaN,NaN,30311.0,7.4,6.2,6.7
4,Africa,Region,2018,2567022.0,2427012.0,1905.0,3.3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640,Zimbabwe,Country,2018,22077.0,24312.0,1684.0,4.8,NaN,NaN,NaN,...,NaN,33.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2641,Zimbabwe,Country,2019,20245.0,21935.0,1498.0,-8.3,NaN,NaN,NaN,...,NaN,31.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2642,Zimbabwe,Country,2020,20407.0,21787.0,1466.0,0.8,97.6,97.1,NaN,...,7.39,31.9,89.1,2870.0,NaN,NaN,391.0,NaN,NaN,NaN
2643,Zimbabwe,Country,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,31.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


((2645, 31),
 Index(['Region', 'Type', 'Year',
        'GDP in constant 2010 prices (millions of US dollars)',
        'GDP in current prices (millions of US dollars)',
        'GDP per capita (US dollars)', 'GDP real rates of growth (percent)',
        'Gross enrollment ratio - Primary (female)',
        'Gross enrollment ratio - Primary (male)',
        'Gross enrollment ratio - Secondary (female)',
        'Gross enrollment ratio - Secondary (male)',
        'Gross enrollment ratio - Upper secondary level (female)',
        'Gross enrollment ratio - Upper secondary level (male)',
        'Labour force participation - Female',
        'Labour force participation - Male',
        'Labour force participation - Total',
        'Population aged 0 to 14 years old (percentage)',
        'Population aged 60+ years old (percentage)', 'Population density',
        'Population mid-year estimates (millions)',
        'Population mid-year estimates for females (millions)',
        'Population mi

In [24]:
df_summary_ = pd.pivot_table(data=df_all, index='Region/Country/Area', columns=['Year', 'Series'], values='Value', aggfunc=np.average)
df_summary_ = pd.merge(df_regions, df_summary_, how='inner',left_on='Region', right_on='Region/Country/Area')
df_summary_.to_csv("out/summary_.csv", index=False)
print(df_summary_.shape, df_summary_.columns)
display(df_summary_[:5])
df_summary_[df_summary_.notnull().all(axis=1)]

(266, 357) Index([                                                                'Region',
                                                                         'Type',
                 (1995, 'GDP in constant 2010 prices (millions of US dollars)'),
                       (1995, 'GDP in current prices (millions of US dollars)'),
                                          (1995, 'GDP per capita (US dollars)'),
                                   (1995, 'GDP real rates of growth (percent)'),
                            (2000, 'Gross enrollment ratio - Primary (female)'),
                              (2000, 'Gross enrollment ratio - Primary (male)'),
                          (2000, 'Gross enrollment ratio - Secondary (female)'),
                            (2000, 'Gross enrollment ratio - Secondary (male)'),
       ...
                           (2022, 'Population aged 60+ years old (percentage)'),
                                                   (2022, 'Population density'),
      

/var/folders/6d/t1t1r89n1q95d2ztv2qstyb40000gr/T/ipykernel_32868/694924819.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  df_summary_ = pd.merge(df_regions, df_summary_, how='inner',left_on='Region', right_on='Region/Country/Area')


,Region,Type,"(1995, GDP in constant 2010 prices (millions of US dollars))","(1995, GDP in current prices (millions of US dollars))","(1995, GDP per capita (US dollars))","(1995, GDP real rates of growth (percent))","(2000, Gross enrollment ratio - Primary (female))","(2000, Gross enrollment ratio - Primary (male))","(2000, Gross enrollment ratio - Secondary (female))","(2000, Gross enrollment ratio - Secondary (male))",...,"(2022, Population aged 60+ years old (percentage))","(2022, Population density)","(2022, Population mid-year estimates (millions))","(2022, Population mid-year estimates for females (millions))","(2022, Population mid-year estimates for males (millions))","(2022, Seats held by women in national parliament, as of February (%))","(2022, Sex ratio (males per 100 females))","(2022, Unemployment rate - Female)","(2022, Unemployment rate - Male)","(2022, Unemployment rate - Total)"
0,Africa,Region,963827.0,588279.0,821.0,2.8,NaN,NaN,NaN,NaN,...,5.5,48.3,1426.74,714.31,712.43,NaN,99.7,8.7,7.4,8.0
1,Americas,Region,15236608.0,10218631.0,13162.0,2.4,NaN,NaN,NaN,NaN,...,17.2,26.7,1037.14,525.34,511.80,NaN,97.4,8.5,6.6,7.4
2,Asia,Region,10031357.0,9320099.0,2668.0,5.5,NaN,NaN,NaN,NaN,...,13.7,150.6,4722.63,2319.78,2402.86,20.2,103.6,NaN,NaN,NaN
3,Australia and New Zealand,Region,762602.0,454246.0,20964.0,4.0,NaN,NaN,NaN,NaN,...,22.5,3.9,31.36,15.79,15.57,39.1,98.6,NaN,NaN,NaN
4,Caribbean,Region,202545.0,121569.0,3450.0,4.2,NaN,NaN,NaN,NaN,...,15.1,200.4,44.39,22.45,21.94,40.9,97.8,11.0,6.6,8.5


,Region,Type,"(1995, GDP in constant 2010 prices (millions of US dollars))","(1995, GDP in current prices (millions of US dollars))","(1995, GDP per capita (US dollars))","(1995, GDP real rates of growth (percent))","(2000, Gross enrollment ratio - Primary (female))","(2000, Gross enrollment ratio - Primary (male))","(2000, Gross enrollment ratio - Secondary (female))","(2000, Gross enrollment ratio - Secondary (male))",...,"(2022, Population aged 60+ years old (percentage))","(2022, Population density)","(2022, Population mid-year estimates (millions))","(2022, Population mid-year estimates for females (millions))","(2022, Population mid-year estimates for males (millions))","(2022, Seats held by women in national parliament, as of February (%))","(2022, Sex ratio (males per 100 females))","(2022, Unemployment rate - Female)","(2022, Unemployment rate - Male)","(2022, Unemployment rate - Total)"


This can be added to a sql db to allow queries

In [25]:
'''df_2020 = df_all[df_all['Year']==2020]
display(df_2020[:5])
df_summary_2020 = pd.pivot_table(data=df_2020, index='Region/Country/Area', columns='Series', values='Value', aggfunc=np.average)
df_summary_2020 = pd.merge(df_regions, df_summary_2020, how='inner',left_on='Region', right_on='Region/Country/Area')
df_summary_2020.to_csv("out/summary_2020.csv", index=False)
print(df_summary_2020.shape, df_summary_2020.columns)
display(df_summary_2020)
df_summary_2020[df_summary_2020.notnull().all(axis=1)]''';


- Data is missing for some regions/countries for some years
- Change in variables across years can't be used 

## Data

| Category | Variable | Description | 
|---|---|---|
| Women in Govt | Seats held by women in national parliament, as of February (%) |  | 
| GDP | GDP in constant 2010 prices (millions of US dollars)  |  | 
| | GDP in current prices (millions of US dollars) |  | 
| | GDP per capita (US dollars) |  | 
| | GDP real rates of growth (percent) | Not clear | 
| Education | Gross enrollment ratio - Primary (female) |  | 
| | Gross enrollment ratio - Primary (male) |  | 
| | Gross enrollment ratio - Secondary (female) |  | 
| | Gross enrollment ratio - Upper secondary level (female) | |
| | Gross enrollment ratio - Upper secondary level (male) | |
| | Students enrolled in primary education (thousands)| |
| | Students enrolled in secondary education (thousands)| |
| | Students enrolled in upper secondary education (thousands)| |
| Population | Population aged 0 to 14 years old (percentage)| |
| | Population aged 60+ years old (percentage)| |
| | Population density| |
| | Population mid-year estimates (millions)| |
| | Population mid-year estimates for females (millions)| |
| | Population mid-year estimates for males (millions)| |
| | Sex ratio (males per 100 females) | |
| Labour | Labour force participation - Total | |
| | Unemployment rate - Total | |
| | Labour force participation - Male | |
| | Unemployment rate - Male | |
| | Labour force participation - Female | |
| | Unemployment rate - Female | |
| | | |

- GDP per capita (US dollars) GDP per capita is gross domestic product divided by midyear population.
- Gross enrollment ratio: Ratio of total enrollment, regardless of age, to the population of the age group that officially corresponds to the level of education shown.
https://databank.worldbank.org/metadataglossary/gender-statistics/series/SE.PRM.ENRR.MA#:~:text=Gross%20enrollment%20ratio%20is%20the,the%20level%20of%20education%20shown.

## New variables

|Variable| What does it represent? |
|---|---|
| (Students enrolled in upper secondary education (thousands) X 1000) / Population mid-year estimates (millions) | A higher value will indicate that more people are educated|
| (GER - Secondary) - (GER - Primary)| A smaller value indicates there is less dropout |
| (GER - Upper Secondary) - (GER - Secondary) | A smaller value indicates there is less dropout |
| (GER - Male) - (GER - Female) | A smaller value indicates there is less gender discrimination|
| [Population aged 60+ years old (percentage)] - [Population aged 0 to 14 years old (percentage)] | If difference is lesser it could mean financial and health condition of the individuals is good. |


### Change in variables across years
It may not be possible to use these as all values are not available for all years
|Variable| What does it represent? |
|---|---|
| Change in Seats held by women across years| |
| Change in GDP per capita across years| |
| Change in Sex ratio (males per 100 females) across years| |


#### Students_in_upper_secondary_education

ALTER TABLE `womeningovt.regiondata.regiondata`
ADD COLUMN `Students_in_upper_secondary_education` FLOAT64;

UPDATE `womeningovt.regiondata.regiondata`
SET `Students_in_upper_secondary_education` = 
IF (`Population_mid_year_estimates__millions_` = 0, 0, 
((`Students_enrolled_in_upper_secondary_education__thousands_` * 1000)/ (`Population_mid_year_estimates__millions_` * 1000000) ) * 100)
WHERE TRUE;

UPDATE `womeningovt.regiondata.regiondata`
SET `Students in upper secondary education` = 0 WHERE `Students_in_upper_secondary_education` is NULL;

UPDATE `womeningovt.regiondata.regiondata`
SET `Students_in_upper_secondary_education` = NULL WHERE `Students_in_upper_secondary_education` = 0.0;

ALTER TABLE `womeningovt.regiondata.regiondata`
DROP COLUMN `Students_in_upper_secondary_education`;

#### Students_in_primary_education
ALTER TABLE `womeningovt.regiondata.regiondata`
ADD COLUMN `Students_in_primary_education` FLOAT64;

UPDATE `womeningovt.regiondata.regiondata`
SET `Students_in_primary_education` = 
IF (`Population_mid_year_estimates__millions_` = 0, 0, 
((`Students_enrolled_in_primary_education__thousands_` * 1000)/ (`Population_mid_year_estimates__millions_` * 1000000) ) * 100)
WHERE TRUE;

UPDATE `womeningovt.regiondata.regiondata`
SET `Students_in_primary_education` = NULL WHERE `Students_in_primary_education` = 0.0;

ALTER TABLE `womeningovt.regiondata.regiondata`
DROP COLUMN `Students_in_primary_education`;

#### Students_in_primary_education_diff

ALTER TABLE `womeningovt.regiondata.regiondata`
ADD COLUMN `Students_in_primary_education_diff` FLOAT64;

UPDATE `womeningovt.regiondata.regiondata`
SET `Students_in_primary_education_diff` =  
(`Gross_enrollment_ratio___Primary__male_` - `Gross_enrollment_ratio___Primary__female_`)
WHERE TRUE;

CREATE TABLE `womeningovt.regiondata.temp` AS SELECT year, region,type, Population_aged_0_to_14_years_old__percentage_ population_0_14, Population_aged_60__years_old__percentage_ population_60, Seats_held_by_women_in_national_parliament__as_of_February____ women_in_govt 
FROM `womeningovt.regiondata.regiondata` WHERE year IN (2000, 2005, 2010, 2015, 2020, 2022)

SELECT CORR(women_in_govt, women_in_govt_lag1)*100 corr_wig1,CORR(women_in_govt, women_in_govt_lag2)*100 corr_wig2, CORR(women_in_govt, women_in_govt_lag3)*100 corr_wig3,
CORR(population_0_14, population_0_14_lag1)*100 corr_p141, CORR(population_0_14, population_0_14_lag2)*100 corr_p142, CORR(population_0_14, population_0_14_lag2)*100 corr_p143,
CORR(population_60, population_60_lag1)*100 corr_p601, CORR(population_60, population_60_lag2)*100 corr_p602, CORR(population_60, population_60_lag3)*100 corr_p603,
 FROM (SELECT year, region, women_in_govt, 
    LAG(women_in_govt, 1) OVER (PARTITION BY region ORDER BY year) women_in_govt_lag1, 
    LAG(women_in_govt, 2) OVER (PARTITION BY region ORDER BY year) women_in_govt_lag2, 
    LAG(women_in_govt, 3) OVER (PARTITION BY region ORDER BY year) women_in_govt_lag3, 
    population_0_14,
    LAG(population_0_14, 1) OVER (PARTITION BY region ORDER BY year) population_0_14_lag1, 
    LAG(population_0_14, 2) OVER (PARTITION BY region ORDER BY year) population_0_14_lag2, 
    LAG(population_0_14, 3) OVER (PARTITION BY region ORDER BY year) population_0_14_lag3, 
    population_60,
    LAG(population_60, 1) OVER (PARTITION BY region ORDER BY year) population_60_lag1,
    LAG(population_60, 2) OVER (PARTITION BY region ORDER BY year) population_60_lag2, 
    LAG(population_60, 3) OVER (PARTITION BY region ORDER BY year) population_60_lag3,
FROM `womeningovt.regiondata.temp`
WHERE type = "Country");